In [1]:
# If not installed, run this cell. It is needed for access to GEE
#!pip install ee
#!pip show ee

In [2]:
# If not installed, run this cell. It helps visualization of data
#!pip install geemap
#!pip show geemap

In [4]:
# Import libraries
import ee, geemap, time

In [5]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jvg')

In [6]:
# HydroSHEDS v1 will be used. Specifically, its HydroBASINS
Map = geemap.Map()
hydrobasins = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_3')
hybas_id = "HYBAS_ID"
Map.addLayer(hydrobasins, {}, 'HydroBasins')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
# Access the asset containing the calibration drainage areas
calib_basins = ee.FeatureCollection('projects/ee-jvg/assets/calibration_basins_gene')
Map.centerObject(calib_basins.geometry())
Map.addLayer(calib_basins, {"color": 'blue'}, 'Calibration Drainage Areas')

In [8]:
# TerraClimate layers filtered with "start_date" (inclusive) and "end_date" (exclusive) and clipped to "zoi"
bands = ["pr", "pet", "ro"]
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').select(bands).filterBounds(calib_basins)
terraclimate.size().getInfo()

792

In [9]:
projection = terraclimate.first().projection().getInfo()

In [10]:
# Let's assign year and month to each image of the resulting image collection
final_ic = terraclimate.map(lambda image: image.set("yr-mth", ee.Image(image).date().get("year").format().cat(ee.String("-")).cat(ee.Image(image).date().get("month").format())))

In [11]:
for band in bands:
    # Define the export parameters
    export_params = {
        'region': calib_basins.geometry(),  # Region of interest
        'fileFormat': 'GeoTIFF',
        'folder': 'GEE_Exports',  # Google Drive folder name
        'crs': projection["wkt"],
        'crsTransform': projection["transform"] ,
        'fileNamePrefix': 'TerraClimate_' + band  # Prefix for the exported file
    }

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(final_ic.select(band).toBands(), **export_params)
    task.start()

    print('Exporting image "' + band + '" to Google Drive...')

    while task.active():
        print('Polling for task (id: {}).'.format(task.id))
        time.sleep(30)

    print('Export completed.')

Exporting image "pr" to Google Drive...
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26JDKUUKGLAHM7IF4AKD).
Polling for task (id: PV2R26J